In [2]:
import os
import numpy as np
import h5py
import rasterio

data_root = './datasets/era5/train/2008.h5'

windninja_root = './datasets/windninja/delos_wind_layer_5070.tif'

In [3]:
with h5py.File(data_root,'r') as f:
    print(f['fields'])
    print(f)

<HDF5 dataset "fields": shape (365, 3, 721, 1440), type "<f4">
<HDF5 file "2008.h5" (mode r)>


In [13]:
raster = rasterio.open(windninja_root)
arr = raster.read()
arr.shape

(1, 42691, 38820)